# Test Develop Mock Circuit

In [1]:
import numpy as np
import pandas as pd
from neuro_dmt.models.bluebrain.circuit.mock.cell import Cell

In [2]:
from neuro_dmt.models.bluebrain.circuit.geometry import Position
p = Position(X = 1., Y = 1., Z = 1.)
print(p.as_tuple)
print(p.value)

random_positions =\
    Position.sample(
        (np.array((-1., -1., -1.)), np.array((1., 1., 1.))),
        n = 3)
print(random_positions)

(1.0, 1.0, 1.0)
[1. 1. 1.]
[[-0.21083023 -0.72427996 -0.13665569]
 [-0.64102257  0.74485763 -0.87864674]
 [ 0.64270505  0.50788733  0.47837893]]


In [3]:
example_cell =\
    Cell(
        layer=1,
        position=Position(X=1., Y=2., Z=3.),
        mtype="L23_BTC")
example_cell.as_dict

{'region': 'brain',
 'layer': 1,
 'nucleus': 'not-defined',
 'mtype': 'L23_BTC',
 'etype': 'not-defined',
 'morph_class': 'not-defined',
 'synapse_class': 'INH',
 'x': 1.0,
 'y': 2.0,
 'z': 3.0}

In [4]:
from neuro_dmt.models.bluebrain.circuit.mock.test_develop\
    import circuit_composition, circuit_connectivity
circuit_composition.cell_density.head()

mean  std
mtype  layer                  
L1_DAC 1      3094.333333  0.0
       2         0.000000  0.0
       3         0.000000  0.0
       4         0.000000  0.0
       5         0.000000  0.0

In [5]:
from neuro_dmt.models.bluebrain.circuit.mock import CircuitBuilder
test_circuit_builder =\
    CircuitBuilder(
        composition=circuit_composition,
        connectivity=circuit_connectivity)
test_circuit_builder.get_cell_density( mtype="L1_DAC", layer=1 )

,,mean,std
mtype,layer,,
L1_DAC,1,3094.333333,0.0


In [6]:
print(circuit_composition.cell_density.loc["L1_DAC"])
print(test_circuit_builder.get_number_cells(1, "L1_DAC"))

              mean  std
layer                  
1      3094.333333  0.0
2         0.000000  0.0
3         0.000000  0.0
4         0.000000  0.0
5         0.000000  0.0
6         0.000000  0.0
23


In [7]:
circuit_cells = test_circuit_builder.get_cells()

In [8]:
import pandas as pd
circuit_cells_df =\
    pd.DataFrame([
        cell.as_dict for cell in circuit_cells])
print("number of cells {}".format(circuit_cells_df.shape[0]))
print(circuit_cells_df.head())

number of cells 12237
         etype  layer  morph_class   mtype      nucleus region synapse_class  \
0  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
1  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
2  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
3  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   
4  not-defined      1  not-defined  L1_DAC  not-defined  brain           INH   

            x            y           z  
0  118.765953  1784.378960   14.790779  
1  125.252217  1792.553843   51.629456  
2  163.759327  1764.745174   49.185442  
3  199.505401  1719.898291  137.067117  
4  208.532731  1795.659227   85.898074  


In [9]:
from neuro_dmt.models.bluebrain.circuit.mock.synapse import Synapse
s = Synapse(pre_gid=1, post_gid=2)
s.as_dict

{'post_gid': 2, 'pre_gid': 1}

In [10]:
cell_collection = test_circuit_builder.get_cell_collection()
cell_collection.get(1, "mtype")

'L1_DAC'

In [11]:
cell_collection.get([1,2], properties=["mtype", "etype"])

,mtype,etype
1,L1_DAC,not-defined
2,L1_DAC,not-defined


In [12]:
from neuro_dmt.models.bluebrain.circuit.mock.circuit import MockCircuit
mock_circuit =\
    MockCircuit.build(
        circuit_composition,
        circuit_connectivity)
mock_conn =\
    mock_circuit.connectome

In [13]:
mock_cells = mock_circuit.cells
mock_cells.get([1, 3000], "mtype")

1       L1_DAC
3000     L4_PC
Name: mtype, dtype: object

In [14]:
mock_circuit.connectome.afferent_gids(2)

array([    0,     5,     6, ..., 12212, 12231, 12234])

In [15]:
mock_circuit.connectome.efferent_gids(3)

array([   12,    16,    20, ..., 12195, 12200, 12229])

In [16]:
mock_circuit.connectome._append_afferent_synapses(2).head()

,post_gid,pre_gid
0,2,0
1,2,0
2,2,0
3,2,0
4,2,0


In [17]:
print(mock_circuit.connectome._append_afferent_synapses(1).head())
print(mock_circuit.connectome._synapses.shape)
print("post_gids in synapses", mock_circuit.connectome._synapses.post_gid.unique())
print("pre_gids in synapses", mock_circuit.connectome._synapses.pre_gid.unique())

   post_gid  pre_gid
0         2        0
1         2        0
2         2        0
3         2        0
4         2        0
(20405, 2)
post_gids in synapses [2 1]
pre_gids in synapses [    0     5     6 ... 12217 12219 12223]


In [18]:
print(np.searchsorted(mock_conn.afferent_adjacency[2][:,0], 1))
mock_conn.afferent_adjacency[1][:,0]

1


array([    0,     4,    13, ..., 12219, 12223, 12234])

In [35]:
print(mock_conn._get_pair_synapse_count(52,1))
print(mock_conn.afferent_adjacency[1][0:10,:])
print(mock_conn.pair_synapses(52, 1))

4
[[ 0  6]
 [ 4  3]
 [13  6]
 [21  6]
 [27  2]
 [30  6]
 [41  6]
 [51  6]
 [52  4]
 [75  4]]
   post_gid  pre_gid
0         1       52
1         1       52
2         1       52
3         1       52


In [ ]:
from dmt.tk.field import Field, WithFields
class AFF(WithFields):
    af = Field(
        """
        AFF af
        """,
        __required__=False)
    baf = Field(
        """
        AFF baf
        """,
        __required__=False)

In [ ]:
aff = AFF(af=1, baf=2, jinga=3)
aff.as_dict

In [ ]:
hasattr(AFF, 'af')

In [ ]:
aff1 = AFF(af = 2)

In [ ]:
hasattr(aff, "af"), hasattr(aff1, "af")

In [ ]:
[f for f in aff.get_fields()
 if hasattr(aff1, f)]

In [ ]:
aff1.as_dict

In [ ]:
from neuro_dmt.models.bluebrain.circuit.mock.synapse import Synapse

In [ ]:
ds = pd.Series([])
ds.index = pd.MultiIndex.from_tuples([], names=["pre_gid", "post_gid"])

In [ ]:
df

In [ ]:
xs = list(range(10))
print(xs)
print(xs[slice(0, 6)])

In [ ]:
xs[slice(10000, 10000)]

In [ ]:
df = pd.DataFrame({
    "x": range(100),
    "y": range(100, 300, 2)
})

In [ ]:
df.iloc[slice(0, 1, 1)]